<a href="https://colab.research.google.com/github/FreeeSoul/DataScience/blob/main/20230221_FasCampus_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!ls -al '/gdrive/MyDrive/data'

total 5402
-rw------- 1 root root  337847 Jul 16  2022 data_202001.csv
-rw------- 1 root root  314682 Jul 16  2022 data_202002.csv
-rw------- 1 root root 4205435 Jul 16  2022 data_2020_1_12.csv
-rw------- 1 root root  652442 Jul 16  2022 data_2020.csv
drwx------ 2 root root    4096 Jul 16  2022 .ipynb_checkpoints
drwx------ 2 root root    4096 Jul 15  2022 TCS_영업소간통행시간_1시간_1개월_202001
drwx------ 2 root root    4096 Jul 15  2022 TCS_영업소간통행시간_1시간_1개월_202002
drwx------ 2 root root    4096 Jul 15  2022 TCS_영업소간통행시간_1시간_1개월_202003
drwx------ 2 root root    4096 Feb 19 04:10 취업캠프


In [39]:
import pandas as pd

file = '/gdrive/MyDrive/data/data_2020_1_12.csv'
data = pd.read_csv(file)
data.head() # 집계시/요일 가지고 407 거리에 대한 통행시간 보다 크냐/작냐로 출력 따질 것임 판단을.

,집계일자,집계시,출발영업소코드,도착영업소코드,통행시간,요일
0,20200101,4,101,105,637,2
1,20200101,4,101,105,773,2
2,20200101,4,101,105,762,2
3,20200101,4,101,105,746,2
4,20200101,4,101,105,875,2


In [40]:
data = data[data['도착영업소코드'] == 140]
data

,집계일자,집계시,출발영업소코드,도착영업소코드,통행시간,요일
292,20200101,6,101,140,15824,2
293,20200101,6,101,140,19623,2
294,20200101,9,101,140,18048,2
295,20200101,17,101,140,17979,2
296,20200101,11,101,140,17349,2
...,...,...,...,...,...,...
152713,20201231,9,101,140,14460,3
152856,20201231,16,101,140,14947,3
152857,20201231,17,101,140,15328,3
152861,20201231,20,101,140,17001,3


In [42]:
data.rename(columns={'도착영업소코드' : '거리'}, inplace=True)
data

,집계일자,집계시,출발영업소코드,거리,통행시간,요일
292,20200101,6,101,140,15824,2
293,20200101,6,101,140,19623,2
294,20200101,9,101,140,18048,2
295,20200101,17,101,140,17979,2
296,20200101,11,101,140,17349,2
...,...,...,...,...,...,...
152713,20201231,9,101,140,14460,3
152856,20201231,16,101,140,14947,3
152857,20201231,17,101,140,15328,3
152861,20201231,20,101,140,17001,3


In [43]:
data_re = data.groupby(['집계시', '요일', '거리'])['통행시간'].mean()
data_re

집계시  요일  거리 
0    0   140    15549.675676
     1   140    15983.102564
     2   140    15129.750000
     3   140    14533.189189
     4   140    15908.275000
                    ...     
23   2   140    16006.125000
     3   140    16118.288889
     4   140    17035.450980
     5   140    16033.622222
     6   140    15297.022727
Name: 통행시간, Length: 168, dtype: float64

In [44]:
data_re = data_re.unstack(level=-1)
data_re

거리               140
집계시 요일              
0   0   15549.675676
    1   15983.102564
    2   15129.750000
    3   14533.189189
    4   15908.275000
...              ...
23  2   16006.125000
    3   16118.288889
    4   17035.450980
    5   16033.622222
    6   15297.022727

[168 rows x 1 columns]

In [45]:
data_out = data_re.reset_index()
data_out

거리,집계시,요일,140
0,0,0,15549.675676
1,0,1,15983.102564
2,0,2,15129.750000
3,0,3,14533.189189
4,0,4,15908.275000
...,...,...,...
163,23,2,16006.125000
164,23,3,16118.288889
165,23,4,17035.450980
166,23,5,16033.622222


In [46]:
stat = data_out.describe()
stat

거리,집계시,요일,140
count,168.000000,168.000000,168.000000
mean,11.500000,3.000000,15781.256504
std,6.942881,2.005979,649.799175
min,0.000000,0.000000,14232.461538
25%,5.750000,1.000000,15272.221805
50%,11.500000,3.000000,15838.602578
75%,17.250000,5.000000,16113.439869
max,23.000000,6.000000,17630.525424


In [47]:
mean = stat[140][1]

In [48]:
# 분류 작업에 대한 정보를 표에 넣어준다.
data_out['Binary'] = data_out[140] >= mean
data_out['Binary'] = data_out['Binary'].map({True : 1, False : 0})
data_out

거리,집계시,요일,140,Binary
0,0,0,15549.675676,0
1,0,1,15983.102564,1
2,0,2,15129.750000,0
3,0,3,14533.189189,0
4,0,4,15908.275000,1
...,...,...,...,...
163,23,2,16006.125000,1
164,23,3,16118.288889,1
165,23,4,17035.450980,1
166,23,5,16033.622222,1


In [49]:
data_list = data_out.values.tolist()
data_list # 집계시(x1)	요일(x2)	통행시간	Binary

[[0.0, 0.0, 15549.675675675675, 0.0],
 [0.0, 1.0, 15983.102564102564, 1.0],
 [0.0, 2.0, 15129.75, 0.0],
 [0.0, 3.0, 14533.18918918919, 0.0],
 [0.0, 4.0, 15908.275, 1.0],
 [0.0, 5.0, 15321.372549019608, 0.0],
 [0.0, 6.0, 14232.461538461539, 0.0],
 [1.0, 0.0, 15200.729166666666, 0.0],
 [1.0, 1.0, 15656.083333333334, 0.0],
 [1.0, 2.0, 15088.617021276596, 0.0],
 [1.0, 3.0, 15112.045454545454, 0.0],
 [1.0, 4.0, 15188.155555555555, 0.0],
 [1.0, 5.0, 15066.396551724138, 0.0],
 [1.0, 6.0, 14635.577777777778, 0.0],
 [2.0, 0.0, 15057.040816326531, 0.0],
 [2.0, 1.0, 15581.347826086956, 0.0],
 [2.0, 2.0, 15345.9375, 0.0],
 [2.0, 3.0, 15154.69642857143, 0.0],
 [2.0, 4.0, 15332.020408163266, 0.0],
 [2.0, 5.0, 14927.155172413793, 0.0],
 [2.0, 6.0, 14741.063829787234, 0.0],
 [3.0, 0.0, 14961.714285714286, 0.0],
 [3.0, 1.0, 15481.681818181818, 0.0],
 [3.0, 2.0, 15498.951219512195, 0.0],
 [3.0, 3.0, 15236.627450980392, 0.0],
 [3.0, 4.0, 15840.645833333334, 1.0],
 [3.0, 5.0, 14882.37037037037, 0.0],
 [3.

In [51]:
x_train = []
y_train = []

for r in data_list:
  x_train.append(r[:2])
  y_train.append(r[-1])

x_train[:5]

[[0.0, 0.0], [0.0, 1.0], [0.0, 2.0], [0.0, 3.0], [0.0, 4.0]]

In [52]:
y_train[:5]

[0.0, 1.0, 0.0, 0.0, 1.0]

#2. 이제 Classification이용해서 모델 형성하기

In [55]:
import tensorflow as tf

learning_rate = 1e-4
learning_epoch = 5000

In [57]:
sgd = tf.keras.optimizers.SGD(learning_rate=learning_rate)
mse = tf.keras.losses.mean_squared_error
model = tf.keras.models.Sequential()

In [58]:
model.add(tf.keras.layers.Dense(1, input_shape=(2, ), activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [60]:
model.fit(x_train, y_train, epochs=learning_epoch)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
6/6 [==============================] - 0s 4ms/step - loss: 0.5581 - accuracy: 0.7202
Epoch 2502/5000
6/6 [==============================] - 0s 3ms/step - loss: 0.5580 - accuracy: 0.7202
Epoch 2503/5000
6/6 [==============================] - 0s 4ms/step - loss: 0.5580 - accuracy: 0.7202
Epoch 2504/5000
6/6 [==============================] - 0s 3ms/step - loss: 0.5579 - accuracy: 0.7202
Epoch 2505/5000
6/6 [==============================] - 0s 5ms/step - loss: 0.5578 - accuracy: 0.7202
Epoch 2506/5000
6/6 [==============================] - 0s 4ms/step - loss: 0.5578 - accuracy: 0.7202
Epoch 2507/5000
6/6 [==============================] - 0s 4ms/step - loss: 0.5577 - accuracy: 0.7202
Epoch 2508/5000
6/6 [==============================] - 0s 4ms/step - loss: 0.5577 - accuracy: 0.7202
Epoch 2509/5000
6/6 [==============================] - 0s 5ms/step - loss: 0.5576 - accuracy: 0.7202
Epoch 2510/5000
6/6 [==============================] - 0s 4ms/step - lo